In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
i = 1
pages = "&page="
link_base = "https://www.lumberjack.com"
scarpe_donna = "/it/shoes?gender=woman"
page = requests.get(link_base+scarpe_donna)
soup = BeautifulSoup(page.text)
links = list()
while i < 14:
    if i < 2:
        product_image = soup.find_all("a", attrs={"data-test": "open-to-product-detail-from-list"})
        for image in product_image:
            link = image.get("href")
            link = link_base + link
            links.append(link)
        i +=1
    else:
        new_url = link_base + scarpe_donna + pages + str(i)
        new_page = requests.get(new_url)
        new_soup = BeautifulSoup(new_page.text)
        new_product_image = new_soup.find_all("a", attrs={"data-test": "open-to-product-detail-from-list"})
        for image in new_product_image:
            link = image.get("href")
            link = link_base + link
            links.append(link)
        i +=1
print(links)

['https://www.lumberjack.com/it/product/evril-sneakers-woman-sd0000000000135509?campaign=c_85&ref_screen_code=category ', 'https://www.lumberjack.com/it/product/evril-sneakers-woman-sd0000000000135510?campaign=c_85&ref_screen_code=category ', 'https://www.lumberjack.com/it/product/athens-sneakers-woman-sd0000000000135512?campaign=c_85&ref_screen_code=category ', 'https://www.lumberjack.com/it/product/ravenne-sneakers-woman-sd0000000000135513?campaign=c_85&ref_screen_code=category ', 'https://www.lumberjack.com/it/product/ravenne-sneakers-woman-sd0000000000135514?campaign=c_85&ref_screen_code=category ', 'https://www.lumberjack.com/it/product/thelma-sneakers-woman-sd0000000000135515?campaign=c_85&ref_screen_code=category ', 'https://www.lumberjack.com/it/product/ravenne-sneakers-woman-sd0000000000135231?campaign=c_85&ref_screen_code=category ', 'https://www.lumberjack.com/it/product/walky-sneakers-woman-sd0000000000135085?campaign=c_85&ref_screen_code=category ', 'https://www.lumberjack

In [4]:
names = list()
for l in links:
    shoes_p = requests.get(l)
    shoes_soup = BeautifulSoup(shoes_p.text)
    names_raw = shoes_soup.find("h1", attrs={"product__name description text-capital text-capitalize order-1 js-product-name"})
    names.append( names_raw.text.strip() )
names

['EVRIL SNEAKERS DONNA',
 'EVRIL SNEAKERS DONNA',
 'ATHENS SNEAKERS DONNA',
 'RAVENNE SNEAKERS DONNA',
 'RAVENNE SNEAKERS DONNA',
 'THELMA SNEAKERS DONNA',
 'RAVENNE SNEAKERS DONNA',
 'WALKY SNEAKERS DONNA',
 'EVRIL SNEAKERS DONNA',
 'EVRIL SNEAKERS DONNA',
 'NAVIGATOR SCARPE DA BARCA DONNA',
 'NAVIGATOR SCARPE DA BARCA DONNA',
 'JOLIE SANDALI DONNA',
 'JOHANNA SANDALI DONNA',
 'JOHANNA SANDALI DONNA',
 'EVRIL SNEAKERS DONNA',
 'WALKY SNEAKERS DONNA',
 'NAVIGATOR SCARPE DA BARCA DONNA',
 'JOLIE SANDALI DONNA',
 'JOLIE SANDALI DONNA',
 'JOLIE SANDALI DONNA',
 'JOHANNA SANDALI DONNA',
 'AGATHA SNEAKERS DONNA',
 'AGATHA SNEAKERS DONNA',
 'FREYA SNEAKERS DONNA',
 'FREYA SNEAKERS DONNA',
 'ROXY SNEAKER DONNA',
 'CECILIA SNEAKERS DONNA',
 'CECILIA SNEAKERS DONNA',
 'DORIS SNEAKER DONNA',
 'HILDA SNEAKER DONNA',
 'HILDA SNEAKER DONNA',
 'HILDA SNEAKER DONNA',
 'HILDA SNEAKER DONNA',
 'FREYA SNEAKERS DONNA',
 'REBECCA SNEAKER DONNA',
 'GAIA SNEAKER DONNA',
 'GAIA SNEAKER DONNA',
 'AGATHA SNEAK

In [5]:
price_r = list()
for l in links:
    shoes_p = requests.get(l)
    shoes_soup = BeautifulSoup(shoes_p.text)
    price_raw = shoes_soup.find("div", attrs={"class":"product__prices"})
    price_r.append( price_raw.text.strip() )
price_r

['64,99\xa0EUR',
 '64,99\xa0EUR',
 '59,99\xa0EUR',
 '39,99\xa0EUR',
 '39,99\xa0EUR',
 '79,99\xa0EUR',
 '39,99\xa0EUR',
 '39,99\xa0EUR',
 '59,99\xa0EUR',
 '59,99\xa0EUR',
 '84,99\xa0EUR',
 '84,99\xa0EUR',
 '47,99\xa0EUR',
 '69,99\xa0EUR',
 '69,99\xa0EUR',
 '64,99\xa0EUR',
 '39,99\xa0EUR',
 '84,99\xa0EUR',
 '47,99\xa0EUR',
 '47,99\xa0EUR',
 '47,99\xa0EUR',
 '69,99\xa0EUR',
 '39,99\xa0EUR',
 '39,99\xa0EUR',
 '49,99\xa0EUR',
 '49,99\xa0EUR',
 '49,99\xa0EUR',
 '69,99\xa0EUR',
 '69,99\xa0EUR',
 '59,99\xa0EUR',
 '89,99\xa0EUR',
 '89,99\xa0EUR',
 '89,99\xa0EUR',
 '89,99\xa0EUR',
 '49,99\xa0EUR',
 '79,99\xa0EUR',
 '69,99\xa0EUR',
 '69,99\xa0EUR',
 '34,99\xa0EUR',
 '59,99\xa0EUR',
 '69,99\xa0EUR',
 '69,99\xa0EUR',
 '69,99\xa0EUR',
 '69,99\xa0EUR',
 '39,99\xa0EUR',
 '39,99\xa0EUR',
 '44,99\xa0EUR',
 '39,99\xa0EUR',
 '39,99\xa0EUR',
 '39,99\xa0EUR',
 '49,99\xa0EUR',
 '84,99\xa0EUR',
 '39,99\xa0EUR',
 '89,99\xa0EUR',
 '59,99\xa0EUR',
 '59,99\xa0EUR',
 '54,99\xa0EUR',
 '49,99\xa0EUR',
 '64,99\xa0EUR